### CHECK CLUSTER STATUS AND OPEN TCP PORT

In [25]:
import pandas as pd
from create_cluster import redshift, ec2, DWH_PORT, DWH_CLUSTER_IDENTIFIER

## First check cluster status
- run this block several times until the cluster status becomes `available`

In [20]:
def prettyRedshiftProps(props):
    '''
    Modifies the output of `redshift.describe_clusters()` into pandas dataframe.
    '''
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])


myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

<ipython-input-20-fabb404eefbd>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cb2lacvxvczc.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-e6fcf09e
7,NumberOfNodes,4


<h3>Take note of the cluster <font color='red'> endpoint and role ARN </font> </h3>
<font color='red'>DO NOT RUN THIS unless the cluster status becomes "available" </font>

In [21]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.cb2lacvxvczc.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::390706518889:role/dwhRole


## Second, open an incoming TCP port to access the cluster ednpoint:

In [26]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[-1]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-b3f104b8')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists
